In [1]:
import json

In [2]:
database_path = '../data/cleaned_vlsp2025_law_db.json'
dataset_path = "../data/VLSP 2025 - MLQA-TSR Data Release/train_data/vlsp_2025_train.json"

with open(database_path, "r") as f:
    database = json.load(f)

with open(dataset_path, "r") as f:
    dataset = json.load(f)

In [3]:
dataset[0]

{'id': 'train_1',
 'image_id': 'train_1_3',
 'question': 'Biển báo cấm xe khách trên 29 chỗ được áp dụng trong các khoảng thời gian nào? ',
 'relevant_articles': [{'law_id': 'QCVN 41:2024/BGTVT', 'article_id': '22'}],
 'question_type': 'Multiple choice',
 'choices': {'A': 'Từ 6:30 đến 8:00 và từ 16:30 đến 18:30; ngoài các khoảng thời gian này không được phép lưu thông.',
  'B': 'Từ 6:30 đến 8:00 và từ 16:30 đến 18:30; ngoài các khoảng thời gian này được phép lưu thông.',
  'C': 'Cấm lưu thông cả ngày.',
  'D': 'D. Không cấm xe khách trên 29 chỗ lưu thông.'},
 'answer': 'B'}

# Construct the dataset json file

## Initialize the text embedding model

In [4]:
import torch
from sentence_transformers import SentenceTransformer

TEXT_MODEL_NAME = "BAAI/bge-m3"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
model = SentenceTransformer(TEXT_MODEL_NAME, device="cpu")

/home/nhhsag12/Documents/Multimodal Machine Learning/VLSP_2025/vlsp_2025_multimodal_rag/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from tqdm import tqdm

for record in tqdm(database, desc="Generating embeddings for the database"):
    pt_embedding = model.encode(record["text"], convert_to_tensor=True)
    np_embedding = pt_embedding.detach().cpu().numpy()
    list_embedding = np_embedding.tolist()
    record["embedding"] = list_embedding


Generating embeddings for the database:   2%|▏         | 6/399 [00:28<30:52,  4.71s/it]


KeyboardInterrupt: 

In [ ]:
dataset_image_base_path = "../data/VLSP 2025 - MLQA-TSR Data Release/train_data/images/"

new_dataset = []
for data in dataset:
    image_file = data["image_id"] + "jpg"

